## Star Tracker ML Pipeline ##

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

print("TensorFlow version:", tf.__version__)

2024-06-27 10:38:41.984862: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 10:38:43.006726: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.16.1


In [2]:
# Configuration values
validation_split = 0.20
input_size = 10
num_epochs = 50
batch_size = 40
saved_model_dir = 'models/'

In [3]:
file = "../images_data/mag5_adverserial_bins/mag5_adverserial_bins.csv"
# TODO: read the header instead of hardcoding it
dataframe = pd.read_csv(file, header=0, names=["HIP", "bin0", "bin1", "bin2", "bin3", "bin4", "bin5", "bin6", "bin7", "bin8", "bin9"])
dataset = dataframe.values
X = dataset[:,1:11].astype(int)
Y = dataset[:,0].astype(int)
print("Input Size:", len(X[0]))
num_classes = len(np.unique(Y))
print("Output Size:", num_classes)

Input Size: 10
Output Size: 1608


In [4]:
# Encode class values as integers and then create dataframe with one hot encoding
# Y_encoded = tf.keras.utils.to_categorical(Y)
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

dummy_Y = tf.one_hot(encoded_Y, depth=num_classes)
print(dummy_Y)



2024-06-26 17:31:54.390475: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-26 17:31:54.446163: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-26 17:31:54.446222: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-26 17:31:54.449832: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-26 17:31:54.449903: I external/local_xla/xla/stream_executor

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(17688, 1608), dtype=float32)


_device.cc:2019] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2024-06-26 17:31:54.600881: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-26 17:31:54.600944: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
# Create the ML Model
star_model = tf.keras.Sequential([
  layers.Dense(input_size, activation="relu", name="layer1"),
  layers.Dense(64, activation="relu", name="layer2"),
  layers.Dense(32, activation="relu", name="layer3"),
  layers.Dense(1608, activation="softmax", name ="layer4")
])

star_model.compile(loss = "categorical_crossentropy",
                      optimizer = 'adam', metrics = ['accuracy'])

In [6]:
# Model Training
star_model.fit(X, dummy_Y, epochs=num_epochs)

Epoch 1/50


I0000 00:00:1719448315.590562   83150 service.cc:145] XLA service 0x7f4d24008f20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719448315.590601   83150 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-06-26 17:31:55.609033: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-26 17:31:55.701220: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1719448316.551553   83207 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 48 bytes spill stores, 48 bytes spill loads

I0000 00:00:1719448316.658773   83213 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 304 bytes spill stores, 304 bytes spill loads


111/553 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0018 - loss: 7.4194  

I0000 00:00:1719448317.519306   83150 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


532/553 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0195 - loss: 6.7670

I0000 00:00:1719448319.206008   83302 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 60 bytes spill stores, 60 bytes spill loads

I0000 00:00:1719448319.222505   83317 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 300 bytes spill stores, 300 bytes spill loads



553/553 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.0216 - loss: 6.7203
Epoch 2/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3701 - loss: 2.4433
Epoch 3/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6272 - loss: 1.2741
Epoch 4/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7660 - loss: 0.8047
Epoch 5/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8381 - loss: 0.5488
Epoch 6/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8778 - loss: 0.4280
Epoch 7/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9038 - loss: 0.3389
Epoch 8/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9200 - loss: 0.2813
Epoch 9/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9183 - loss: 0.2848
Epoch 10/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9312 - loss: 0.2358
Epoch 11/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9296 - loss: 0.2441
Epoch 12/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

In [7]:
# Save Model

star_model.save(saved_model_dir + 'star_tracker_model_large.keras')

# TODO: tflite conversion is not working.
# TODO: figure out how to export tags too
# # Convert the model
# converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel directory
# tflite_model = converter.convert()

# # Save the model.
# with open(saved_model_dir + 'star_tracker_model_small.tflite', 'wb') as f:
#   f.write(tflite_model)

NameError: name 'star_model' is not defined